Source: https://dev.to/ethand91/simple-age-and-gender-detection-using-python-and-opencv-319h

https://github.com/smahesh29/Gender-and-Age-Detection

In [ ]:
#!pip install agegender_estimation

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement agegender_estimation (from versions: none)
ERROR: No matching distribution found for agegender_estimation


In [1]:
import numpy as np
import os, cv2
from google.colab.patches import cv2_imshow
import pandas as pd
import math
import sys
#from agegender_estimation import AgeGenderEstimation

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!git clone https://github.com/smahesh29/Gender-and-Age-Detection.git

fatal: destination path 'Gender-and-Age-Detection' already exists and is not an empty directory.


In [4]:
# Defined the model files
FACE_PROTO = "/content/Gender-and-Age-Detection/opencv_face_detector.pbtxt"
FACE_MODEL = "/content/Gender-and-Age-Detection/opencv_face_detector_uint8.pb"

AGE_PROTO = "/content/Gender-and-Age-Detection/age_deploy.prototxt"
AGE_MODEL = "/content/Gender-and-Age-Detection/age_net.caffemodel"

GENDER_PROTO = "/content/Gender-and-Age-Detection/gender_deploy.prototxt"
GENDER_MODEL = "/content/Gender-and-Age-Detection/gender_net.caffemodel"

# Load network
FACE_NET = cv2.dnn.readNet(FACE_MODEL, FACE_PROTO)
AGE_NET = cv2.dnn.readNet(AGE_MODEL, AGE_PROTO)
GENDER_NET = cv2.dnn.readNet(GENDER_MODEL, GENDER_PROTO)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
AGE_LIST = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
GENDER_LIST = ["Male", "Female"]

box_padding = 20


In [5]:
def get_face_box (net, frame, conf_threshold = 0.5):
  frame_copy = frame.copy()
  frame_height = frame_copy.shape[0]
  frame_width = frame_copy.shape[1]
  blob = cv2.dnn.blobFromImage(frame_copy, 1.0, (300, 300), [104, 117, 123], True, False)

  net.setInput(blob)
  detections = net.forward()
  boxes = []

  for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > conf_threshold:
      x1 = int(detections[0, 0, i, 3] * frame_width)
      y1 = int(detections[0, 0, i, 4] * frame_height)
      x2 = int(detections[0, 0, i, 5] * frame_width)
      y2 = int(detections[0, 0, i, 6] * frame_height)
      boxes.append([x1, y1, x2, y2])
      cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), int(round(frame_height / 150)), 8)

  return frame_copy, boxes

In [6]:
def age_gender_detector (input_path, filename):
  image = cv2.imread(input_path)
  resized_image = cv2.resize(image, (640, 480))

  frame = resized_image.copy()
  frame_face, boxes = get_face_box(FACE_NET, frame)

  count = 0
  for box in boxes:
    frameName.append(filename)
    person.append(count)
    boxFace.append(box)
    face = frame[max(0, box[1] - box_padding):min(box[3] + box_padding, frame.shape[0] - 1), \
      max(0, box[0] - box_padding):min(box[2] + box_padding, frame.shape[1] - 1)]

    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB = False)
    GENDER_NET.setInput(blob)
    gender_predictions = GENDER_NET.forward()
    gender = GENDER_LIST[gender_predictions[0].argmax()]
    gen.append(gender)
    gender_conf.append(gender_predictions[0].max())

    AGE_NET.setInput(blob)
    age_predictions = AGE_NET.forward()
    age = AGE_LIST[age_predictions[0].argmax()]
    ages.append(age)
    age_conf.append(age_predictions[0].max())

    label = "{},{}".format(gender, age)
    cv2.putText(frame_face, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame_face, str(count), (box[0] + 2, box[1] + 15), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA, )

    count += 1

  if len(boxes) > 0:
    cv2.resize(frame_face, (640, 480))
    cv2.imwrite("/content/gdrive/MyDrive/digital_humanities/Ergebnisse/Agender/Siedler/" + filename, frame_face)

  return frame_face

In [7]:
frameName = []
person = []
boxFace = []
gen = []
gender_conf = []
ages = []
age_conf = []

In [8]:
directory = os.fsencode("/content/gdrive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/")
count = 0
for file in os.listdir(directory):
  count += 1
  if count == 100:
    break
  
  filename = os.fsdecode(file)
  age_gender_detector("/content/gdrive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/" + filename, filename)
  

KeyboardInterrupt: ignored

In [ ]:
df = pd.DataFrame(list(zip(frameName, person, boxFace, gen, gender_conf, ages, age_conf)))
df.columns = ['Name', 'person', 'box', 'gender', 'gender_conf', 'age', 'age_conf']
df.to_csv("/content/gdrive/MyDrive/digital_humanities/Ergebnisse/Agender/siedler_agender.csv")